In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import os
import zipfile
import requests
from sklearn.preprocessing import MinMaxScaler
!pip install fuzzywuzzy python-Levenshtein
from fuzzywuzzy import process
import glob

### Descomprimir archivos .zip

In [ ]:
directorio_zip = '/content/*.zip'
files = glob.glob(directorio_zip)

for archivo_zip in files:
  with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
      zip_ref.extractall('/content/')

In [ ]:
# Barrios de Los Ángeles
gdf_barrios = gpd.read_file('/content/Data2/Neighborhood_Councils.geojson')
gdf_barrios = gdf_barrios[['NAME', 'geometry']]
gdf_barrios.columns = ['Neighborhood', 'geometry']
gdf_barrios

In [ ]:
# Read Crime Data csv
df_crimenes = pd.read_csv('/content/Data2/Crime_Data_from_2020_to_Present_20251122.csv')

# Conseguimos datos de longitud y latitud válidos del dataset de crímenes
df_crimenes = df_crimenes[
    (df_crimenes['LAT'] != 0) &
    (df_crimenes['LON'] != 0) &
    (df_crimenes['LAT'].notnull())
].copy()
print(f"Procesando {len(df_crimenes)} crímenes con coordenadas válidas...")

# Convertimos de DataFrame a GeoFrame, el df de crímenes
gdf_crimenes = gpd.GeoDataFrame(df_crimenes, geometry=gpd.points_from_xy(df_crimenes['LON'], df_crimenes['LAT'],
                                                                         crs='EPSG:4326'))
# Verificamos sistema de coordenadas (ambos tienen que estar en el mismo para poder trabajar sobre ellos)
if gdf_barrios.crs != gdf_crimenes.crs:
  gdf_barrios = gdf_barrios.to_crs("EPSG:4326")

# Spatial join (para poder identificar cada crímen con el barrio mediante las coordenadas).
# La variable tiene todas las columnas de crímenes junto con el barrio
crimenes_barrios = gpd.sjoin(gdf_crimenes, gdf_barrios, how='left', predicate='within')

# Resultados para tener el conteo
num_crimenes_barrio = crimenes_barrios['Neighborhood'].value_counts().reset_index()
num_crimenes_barrio.columns = ['Neighborhood', 'Crimenes_Barrio']
num_crimenes_barrio.head()



In [ ]:
# Precio de compra de vivienda en Los Ángeles
df_home_value = pd.read_csv('/content/Data1/Precio_Compra_LA.csv')
df_home_value = df_home_value [
    (df_home_value['City'] == 'Los Angeles') &
    (df_home_value['State'] == 'CA')
].copy()

# Identificar columnas con las que haremos el promedio de valores
cols_24_25 = [col for col in df_home_value.columns if '2024' in col or '2025' in col]

# Calculamos el promedio del precio de compra de esos años
df_home_value['Promedio_precio'] = df_home_value[cols_24_25].mean(axis=1)

# Nos quedamos con las columnas necesarias
df_home_value = df_home_value[['RegionName', 'Promedio_precio']]
df_home_value.columns = ['Neighborhood', 'Avg_price']
df_home_value.reset_index()

# Densidad de población Los Ángeles

In [ ]:
# Cargamos el mapa de California y lo limpiamos (shapefile)
# Primero descomprimimos el archivo ZIP
with zipfile.ZipFile("tl_2023_06_tract.zip", 'r') as zip_ref:
    zip_ref.extractall()

gdf_mapa = gpd.read_file('tl_2023_06_tract.shp')

# Nos quedamos solo con Los Ángeles
gdf_mapa_LA = gdf_mapa[
    (gdf_mapa['COUNTYFP'] == '037') &
    (gdf_mapa['STATEFP'] == '06')
].copy()
#print(f"Polígonos cargados: {len(gdf_mapa_LA)}")

# Cargamos los datos de la población de Los Ángeles
df_population = pd.read_csv('/content/Data1/Population_LA.csv', header=1)

# Renombramos las columnas
df_population = df_population.rename(columns={
    df_population.columns[0]: 'GEOID',
    df_population.columns[2]: 'Total_Population'
})

# Limpiamos el ID para eliminar la parte "1400000US" del ID_Cens
df_population['GEOID'] = df_population['GEOID'].str.replace('1400000US', '')

# Merge para realizar la intersección de los datos y sacar la población por barrio, unimos mediante 'Geometry_ID_Cens'
gdf_population = gdf_mapa_LA.merge(df_population, on='GEOID', how='inner')

# Calculamos la densidad (personas / km2)
gdf_population = gdf_population.to_crs("EPSG:26911") # Representamos la proyección de las coordenadas según la franja de Los Ángeles (UTM Zona 11N)
gdf_population['Area_km2'] = gdf_population.area / 10**6 # Para que sean km2
gdf_population['Densidad_Poblacion'] = gdf_population['Total_Population'] / gdf_population['Area_km2']

#print(gdf_population[['GEOID', 'Total_Population', 'Densidad_Poblacion']].head())

# Relacionamos la densidad de población de cada ID Geometrico con el barrio de LA que corresponda
gdf_population = gdf_population.to_crs("EPSG:26911")
gdf_barrios = gdf_barrios.to_crs("EPSG:26911")

# Calculamos el tamaño del territorio para poder relacionarlo con los barrios
gdf_population['area_tract_original'] = gdf_population.geometry.area

# Hacemos el cálculo de la intersección. Si un tramo cae entre dos barrios, se generan dos filas
gdf_intersec = gpd.overlay(gdf_barrios, gdf_population, how='intersection')

# Cálculo del área de la nueva intersección
gdf_intersec['area_interseccion'] = gdf_intersec.geometry.area

# Cálculo del ratio del tramo obtenido para saber qué porcentaje pertenece al tramo original
gdf_intersec['ratio'] = gdf_intersec['area_interseccion'] / gdf_intersec['area_tract_original']

# Asignación de población proporcional (según el número de personas y el ratio)
gdf_intersec['asignacion_poblacion'] = gdf_intersec['Total_Population'] * gdf_intersec['ratio']

# Resultados por barrio
# Agrupamos tramos para tener el total por barrio
df_demografia_barrio = gdf_intersec.groupby('Neighborhood')[['asignacion_poblacion', 'area_interseccion']].sum().reset_index()

# Convertimos área a km2
df_demografia_barrio['Area_km2'] = df_demografia_barrio['area_interseccion'] / 10**6

# Calculamos densidad final
df_demografia_barrio['Densidad_Poblacion'] = df_demografia_barrio['asignacion_poblacion'] / df_demografia_barrio['Area_km2']
df_demografia_barrio.head()


# Estilo de vida y servicios y mobilidad y transporte en LA

In [ ]:
# Buscamos en el área de 'Los Angeles'
# nwr = node, way, relation (buscamos puntos, líneas y polígonos)

overpass_query = """
[out:json][timeout:90];
area["name"="Los Angeles"]["admin_level"="8"]->.searchArea;
(
  // ESTILO DE VIDA (Comida, Ocio...)
  nwr["amenity"="restaurant"](area.searchArea);
  nwr["amenity"="cafe"](area.searchArea);
  nwr["amenity"="bar"](area.searchArea);
  nwr["leisure"="park"](area.searchArea);
  nwr["leisure"="fitness_centre"](area.searchArea);

  // SERVICIOS
  nwr["amenity"="hospital"](area.searchArea);
  nwr["amenity"="school"](area.searchArea);
  nwr["shop"="supermarket"](area.searchArea);
  nwr["public_transport"="station"](area.searchArea);
);
out center;
"""

# URL de la API pública
url = "http://overpass-api.de/api/interpreter"
response = requests.get(url, params={'data': overpass_query})

if response.status_code == 200:
    data = response.json()
    elements = data['elements']

    amenities_list = []

    for element in elements:
        # Extraemos los tags
        tags = element.get('tags', {})

        # Determinamos la categoría principal
        category = "Other"
        if 'amenity' in tags:
            category = tags['amenity']
        elif 'leisure' in tags:
            category = tags['leisure']
        elif 'shop' in tags:
            category = tags['shop']
        elif 'public_transport' in tags:
            category = 'public_transport'

        # Obtenemos latitud/longitud (si es un 'way' o 'relation', usamos el centro calculado por la API)
        lat = element.get('lat') or element.get('center', {}).get('lat')
        lon = element.get('lon') or element.get('center', {}).get('lon')

        if lat and lon:
            amenities_list.append({
                'Name': tags.get('name', 'Unknown'),
                'Category': category,
                'Type': element['type'],
                'LAT': lat,
                'LON': lon
            })

    # Crear DataFrame
    df_servicios = pd.DataFrame(amenities_list)
    df_servicios = df_servicios[~df_servicios['Category'].isin(['community_centre', 'ferry_terminal'])]
    print("\nResumen por Categoría:")
    print(df_servicios['Category'].value_counts())

else:
    print(f"Error en la consulta: {response.status_code}")

### Agrupamos por barrios de LA

In [ ]:
# Transformamos a GeoDataFrame
gdf_servicios = gpd.GeoDataFrame(df_servicios,
                                 geometry=gpd.points_from_xy(df_servicios['LON'], df_servicios['LAT']),
                                 crs="EPSG:4326") # Devolvemos coordenadas en grados

# Comprobamos crs de ambos gdfs.
if gdf_servicios.crs != gdf_barrios.crs:
    gdf_servicios = gdf_servicios.to_crs(gdf_barrios.crs)

# Spatial join --> Aqui tendremos una fila por cada servicio y barrio
servicios_barrios = gpd.sjoin(gdf_servicios, gdf_barrios, how='inner', predicate='within')

# Contamos por categoria para agrupar en cada barrio
servicios = pd.crosstab(servicios_barrios['Neighborhood'], servicios_barrios['Category'])
servicios = servicios.reset_index()
servicios

# FuzzyWuzzy
### Aplicamos esta técnica para tener los mismos nombres de neighborhood que en el df original cargados del geojson, ya que, no coindician.

In [ ]:
# Obtenemos la lista original de tus barrios
barrios_oficiales = gdf_barrios['Neighborhood'].unique()

# Preparamos el diccionario de precios de Zillow para buscar rápido
dict_precios = pd.Series(
    df_home_value.Avg_price.values,
    index=df_home_value.Neighborhood
).to_dict()

nombres_zillow = list(dict_precios.keys())

# Algoritmo de emparejamiento (Fuzzy Matching)
mapa_precios = {}

for barrio in barrios_oficiales:
    # Busca el nombre más parecido en la lista de Zillow
    match = process.extractOne(barrio, nombres_zillow)

    if match:
        nombre_encontrado, score = match
        # Si la similitud es alta (>80%), cogemos el precio
        if score >= 80:
            mapa_precios[barrio] = dict_precios[nombre_encontrado]
        else:
            mapa_precios[barrio] = None
    else:
        mapa_precios[barrio] = None

# Sobreescribimos df_home_value con los nombres de neighborhood correcto
df_home_value = pd.DataFrame(list(mapa_precios.items()), columns=['Neighborhood', 'Avg_price'])

# Rellenamos los barrios que no se encontraron con el precio promedio del barrio
promedio_global = df_home_value['Avg_price'].mean()
df_home_value['Avg_price'] = df_home_value['Avg_price'].fillna(promedio_global)

df_home_value.head()

# DataFrame final para representarlo en el frontend

In [ ]:
# Juntamos las columnas necesarias de los anteriores dataframes en uno final que contenga todos los features necesarios
# para poder hacer la elección de barrio en base al input del usuario.
df_final = gdf_barrios[['Neighborhood']].copy()

# Añadimos la densidad de población
df_final = df_final.merge(
    df_demografia_barrio[['Neighborhood', 'Densidad_Poblacion']],
    on = 'Neighborhood',
    how = 'left'
)

# Añadimos los servicios y transportes
df_final = df_final.merge(
    servicios,
    on = 'Neighborhood',
    how = 'left'
)

# Añadimos los crimenes
df_final = df_final.merge(
    num_crimenes_barrio,
    on = 'Neighborhood',
    how = 'left'
)

# Añadimos el precio de la vivienda
df_final = df_final.merge(
    df_home_value,
    on = 'Neighborhood',
    how = 'left'
)
df_final


# Normalización y escalado de los datos

In [ ]:
# Buscamos todo lo que sea número (int, float) automáticamente
columnas_numericas = df_final.select_dtypes(include=[np.number]).columns.tolist()
cols_sin_escalar = ['Neighborhood']
cols_a_escalar = [col for col in columnas_numericas if col not in cols_sin_escalar]

# Inicializamos el scaler
scaler = MinMaxScaler()

# Normalizamos los datos
df_final[cols_a_escalar] = scaler.fit_transform(df_final[cols_a_escalar])
df_final['security'] = 1 - df_final['Crimenes_Barrio']

# Printamos el dataframe
df_final

# Exportamos como csv el dataframe final
df_final.to_csv('model_data.csv', index=True, sep=';')